![CSI 702 logo](/img/course-logo-full.png)

# Can we optimize the particle simulation using a simple sort?

---

![CC BY-SA 4.0 license](/img/cc-by-sa.png)

This notebook is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/).

## Load packages

In [1]:
%matplotlib inline
%load_ext Cython

import warnings

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

import autograder

warnings.filterwarnings('ignore')

## Prelude: Let's wrap our particles simulation C++ source code using Cython

### Overview

C++ is the primary language that we are using for this course, and it is worth your time to become familiar enough with the language as it is used in a lot of performance-critical code. The key phrase here is **performance-critical code**, C++ should be utilized when you encounter bottlenecks that cannot be addressed via other means. However, for any given scientific application, you will find that only a fraction of the total code base is dedicated to computations that need to be highly optimized. If this is the case, then what is the rest of the code doing? While each application is different, I wager that you'll still find that a significant portion of scientific code will be dedicated to defining a user interface (sometimes referred to as the API) and with completing tasks associated with data preprocessing and postprocessing. These are tasks that are called only a few times during a run, and, relative to the computational sections of the code, do not take long to execute. Yet, writing these sections of code in C++ can be very tedious and time-consuming.

In some ways, Python is like the polar opposite of C++. Thanks to Python's standard library and a rich ecosystem of external libraries---in scientific computing, projects such as `numpy`, `scipy`, `matplotlib`, `pandas`, `sympy`, `scikit-learn`, etc. are invaluable---it is (relatively) easy to construct simple user interfaces and data preprocessing and postprocessing pipelines. However, "vanilla" Python is slow when you you need to perform an expensive scientific calculation. This can be mitigated by `numpy` and `pandas` if your calculation is amenable to vectorization. However, if the loops in your calculation depend on one another, such as time-dependent simulations or computations that require the use of an iterative solver, then Python's limitations quickly become apparent.

So perhaps you're thinking at this point that it would be nice if we could construct a high-level interface and data preprocessing and postprocessing sections in Python and then write the performance critical sections in C++. I agree, it would be very nice, and even better, it **is** possible to do. And, thanks to the [Cython project](http://docs.cython.org), it isn't that hard to do.

### What is Cython?

> [[Cython]](http://docs.cython.org/en/latest/src/quickstart/overview.html#cython) is a programming language that makes writing C extensions for the Python language as easy as Python itself. It aims to become a superset of the [Python](http://docs.cython.org/en/latest/src/quickstart/overview.html#python) language which gives it high-level, object-oriented, functional, and dynamic programming. Its main feature on top of these is support for optional static type declarations as part of the language. The source code gets translated into optimized C/C++ code and compiled as Python extension modules. This allows for both very fast program execution and tight integration with external C libraries, while keeping up the high programmer productivity for which the Python language is well known.
>
> —[From the Cython documentaion](http://docs.cython.org/en/latest/src/quickstart/overview.html#cython-an-overview)

### Case study: particles simulation

Cython is one of those tools that is easier to understand after you've used it a few times. So, let's step through what we need to do to replace `cli.cpp` with a Cython-based interface. We begin by putting together a Cython description file that generates an interface to the `cli_parameters_t` struct in [common.hpp](common.hpp) and the `serial()` function declared in [serial.cpp](serial.cpp). The file [particles.pxd](particles.pxd) looks as follows:

```cython
from libcpp.string cimport string

cdef extern from "common.hpp":
    cdef cppclass cli_parameters_t:
        cli_parameters_t() except +
        int number_particles
        string output_filename
        string summary_filename
        bint disable_checks
        string benchmark_mode


cdef extern from "serial.hpp" namespace "serial":
    string serial(cli_parameters_t &cli_parameters)
```

It looks a lot like a C/C++ header file, doesn't it? It serves a similar purpose to a header file in Cython.

Next, we will need to define a Cython class that wraps our C++ code, generates a `cli_parameters_t` struct similar to `cli.cpp`, and can be instantiated and called from within Python. Conveniently, we can do this within our Jupyter notebook provided we run `%load_ext Cython` first (which we have). The code that wraps our particle simulation code is provided below:

In [2]:
%%cython --cplus --include=. -c=-fno-wrapv -c=-O3 --force

from libcpp.string cimport string
from particles cimport cli_parameters_t, serial


cdef extern from "common.cpp":
    pass

cdef extern from "serial.cpp" namespace "serial":
    pass


cdef class ParticleSim:
    cdef cli_parameters_t c_parameters
    
    def __cinit__(
        self,
        int number_particles = 1000,
        string output_filename = b"",
        string summary_filename = b"",
        bint disable_checks = False,
        string benchmark_mode = b"naive",
    ):
        self.c_parameters = cli_parameters_t()
        self.c_parameters.number_particles = number_particles
        self.c_parameters.output_filename = output_filename
        self.c_parameters.summary_filename = summary_filename
        self.c_parameters.disable_checks = disable_checks
        self.c_parameters.benchmark_mode = benchmark_mode
        
    def run(self):
        cdef string results_string
        results_string = serial(self.c_parameters)
        print(results_string.decode())
        
    @property
    def number_particles(self):
        return self.c_parameters.number_particles
    
    @number_particles.setter
    def number_particles(self, value):
        self.c_parameters.number_particles = value
    
    @property
    def output_filename(self):
        return self.c_parameters.output_filename
    
    @output_filename.setter
    def output_filename(self, value):
        self.c_parameters.output_filename = value
    
    @property
    def summary_filename(self):
        return self.c_parameters.summary_filename
    
    @summary_filename.setter
    def summary_filename(self, value):
        self.c_parameters.summary_filename = value
    
    @property
    def disable_checks(self):
        return self.c_parameters.disable_checks

    @disable_checks.setter
    def disable_checks(self, value):
        self.c_parameters.disable_checks = value
    
    @property
    def benchmark_mode(self):
        return self.c_parameters.benchmark_mode

    @benchmark_mode.setter
    def benchmark_mode(self, value):
        self.c_parameters.benchmark_mode = value

To understand what's going on, let's break the code down into a smaller chunks. The first line,

```cython
%%cython --cplus --include=. -c=-fno-wrapv -c=-O3 --force
```

is a [*magic* command](https://ipython.readthedocs.io/en/stable/interactive/magics.html) that sets up the code cell so that it can be transpiled into C++ by Cython and subsequently compiled using the GNU compilers. `--cplus` states that we want the transpiled code to be C++ and not C code. `--include=.` lets Cython look for header files in the same directory as this notebook. The two `-c` options are compiler flags. `--force` means that Cython will regenerate and compile the code each time the block is run, regardless of whether or not anything has changed.

The next part contains our Cython-specific imports,

```cython
from libcpp.string cimport string
from particles cimport cli_parameters_t, serial
```

Note that when we're importing Cython-specific modules, we use `cimport` instead of `import`. Any module included using `cimport` can only be used by C/C++ objects and interfaces, regular Python cannot interact with these modules. `from libcpp.string cimport string` imports an interface to `std::string`, and `from particles cimport cli_parameters_t, serial` imports the interface to the `cli_parameters_t` datatype and `serial` function we declared in `particles.pxd`.

The next part tells Cython to include and compile the C++ code in `common.cpp` and `serial.cpp` as part of our program:

```cython
cdef extern from "common.cpp":
    pass

cdef extern from "serial.cpp" namespace "serial":
    pass
```

The code block starting with `cdef class` defines the class `ParticleSim` that is the interface between Python and the C++ code. These Python and C/C++ interfaces have a special initialization method called `__cinit__`, which serves a similar purpose to `__init__` in regular Python class definitions.

```cython
cdef class ParticleSim:
    cdef cli_parameters_t c_parameters
    
    def __cinit__(
        self,
        int number_particles = 1000,
        string output_filename = b"",
        string summary_filename = b"",
        bint disable_checks = False,
        string benchmark_mode = b"naive",
    ):
        self.c_parameters = cli_parameters_t()
        self.c_parameters.number_particles = number_particles
        self.c_parameters.output_filename = output_filename
        self.c_parameters.summary_filename = summary_filename
        self.c_parameters.disable_checks = disable_checks
        self.c_parameters.benchmark_mode = benchmark_mode
        
    def run(self):
        cdef string results_string
        results_string = serial(self.c_parameters)
        print(results_string.decode())
```

Note that we instantiate a `cli_parameters_t` struct named `c_parameters`. This name needs to match the class attribute `self.c_parameters`. The `string` datatype (remember, this is equivalent to `std::string`) only accepts byte-encoded Python strings. If you try to pass a unicode string, you will receive an error. The `run()` method is what will be used to run the particle simulation itself.

The rest of the class definition is used to expose the different members of the `cli_parameters_t` struct so that they can be accessed and changed by Python.

```cython
    @property
    def number_particles(self):
        return self.c_parameters.number_particles
    
    @number_particles.setter
    def number_particles(self, value):
        self.c_parameters.number_particles = value
    
    @property
    def output_filename(self):
        return self.c_parameters.output_filename
    
    @output_filename.setter
    def output_filename(self, value):
        self.c_parameters.output_filename = value
    
    @property
    def summary_filename(self):
        return self.c_parameters.summary_filename
    
    @summary_filename.setter
    def summary_filename(self, value):
        self.c_parameters.summary_filename = value
    
    @property
    def disable_checks(self):
        return self.c_parameters.disable_checks

    @disable_checks.setter
    def disable_checks(self, value):
        self.c_parameters.disable_checks = value
    
    @property
    def benchmark_mode(self):
        return self.c_parameters.benchmark_mode

    @benchmark_mode.setter
    def benchmark_mode(self, value):
        self.c_parameters.benchmark_mode = value
```

To run the code, we instantiate the `ParticleSim` class,

In [3]:
particles = ParticleSim()

And then run the simulation:

In [4]:
particles.run()

n = 1000, simulation time = 2.11941 seconds, absmin = 0.778821, absavg = 0.957902



## A simple example of optimizing the particle simulation

**Note:** Simply taking and using the code example here as your submission for Homework 2 will result in a severe penalty to the write-up portion of your Homework 2, Part A grade!

Now that we have a Cython wrapper for our C++ code, let's consider an implementation one might try as a first attempt when solving Homework 2, Part A. This idea comes from a simple collision detection method used in video games called the [*Axis-Aligned Bounding Box* (AABB) algorithm](https://developer.mozilla.org/en-US/docs/Games/Techniques/2D_collision_detection). The basic idea is to surround each object with a rectangle and then, for a given object, check whether any portion of its rectangle overlaps with the rectangle of another object. This idea is illustrated in the figure below.

![](/img/particles_aabb_schematic.png)

If they do overlap, then there might be a collision and a more accurate calculation is needed. If they don't, then it is not possible for there to be a collision and no further checking is required.

We can apply this idea if we start by sorting our vector of particles (datatype `std::vector<particle_t>`) along the x-axis. With this, we will know that, for a given particle $i$, any particles adjacent to element $i$ in the vector will be close along the *x* dimension.

$$
\begin{gathered}
\text{particles} \to
\left[
\begin{array}{c|c|c|c|c|c|c}
\dots & i-2 & i - 1 & i & i + 1 & i + 2 & \dots
\end{array}
\right]
\end{gathered}
$$

Thus, there is a greater liklihood that particles $i-1$ and $i+1$ will potentially interact with particle $i$, while there is a much slimmer chance that particles $i-100$ and $i+100$ will interact. Since we know that the interaction radius for each particle is always 0.01 units, then after the sort, for a given particle $i$, we only need to check the particles in the range $\text{particle[i].x} - 0.01 < x < \text{particle[i].x} + 0.01$. The figure below illustrates what this looks like in practice:

![](/img/particles_sort_schematic.png)

In the above figure, the orange particle is particle $i$. The semi-transparent orange rectangle is the range $\text{particle[i].x} - 0.01 < x < \text{particle[i].x} + 0.01$. So, we need only check the blue particles that overlap with the orange rectangle, which will reduce the number of checks per particle in each time step. 


Implementing the sort in C++ requires us to declare the following struct,

```cpp
struct {
  bool operator()(const particle_t &a, const particle_t &b) const
  {   
    return a.x < b.x;
  }   
} particles_x_sort;
```

This struct will be provided as an input to a sorting function. This struct behaves like a C++ class, and defines how we want to sort the `particle_t` datatype within our `particles` vector. It takes two elements, `a` and `b`, inspects the `.x` attribute of each, and checks if `a.x < b.x`. Whenever this is false, the list is updated accordingly.

To sort our `particles` vector, we use,

```cpp
std::sort(particles.begin(), particles.end(), particles_x_sort);
```

After calling this function, the `particles` vector will be sorted according to the rule we defined in the `particles_x_sort` struct.

To make use of the sorted `particles` vector, we modify the compute forces section of `serial.cpp` as follows,

```cpp
for (int i = 0; i < n; i++) {
  int j_forwards = i + 1;
  int j_backwards = i - 1;
  double x_pos_forwards = particles[i].x + 0.01;
  double x_pos_backwards = particles[i].x - 0.01;
  
  particles[i].ax = 0.0;
  particles[i].ay = 0.0;
  
  if (j_forwards < n) {
    while (j_forwards < n && x_pos_forwards > particles[j_forwards].x) {
      apply_force(particles[i], particles[j_forwards], dmin, davg, navg);
      j_forwards++;
    }
  }
  
  if (j_backwards >= 0) {
    while (j_backwards >= 0 && particles[j_backwards].x > x_pos_backwards) {
      apply_force(particles[i], particles[j_backwards], dmin, davg, navg);
      j_backwards--;
    }
  }
}
```

The above code starts at `particles[i]` and scans forwards and backwards in the `particles` vector. The `if` statements check if we are already at the beginning or end of the vector. If not, then the scan proceeds and checks if an adjacent vector element is within the range $\text{particle[i].x} - 0.01 < x < \text{particle[i].x} + 0.01$. If it is, then the `apply_force` function is called. If it is not, then we know we've reached the range boundary and the scan stops.

To test that everything is working, we instantiate two `ParticleSim` objects, one that runs in the naive $O(n^2)$ mode and one that runs in the one-dimensional sort mode we just implemented.

In [5]:
particles_sort = ParticleSim(benchmark_mode=b"sort")
particles_naive = ParticleSim(benchmark_mode=b"naive")

We benchmark the sorting method for simulations containing 4000, 8000, 16000, 32000, and 64000 particles.

In [7]:
particles_sort.summary_filename = b"particles_sort_summary.txt"
for n in [4000, 8000, 16000, 32000, 64000]:
    particles_sort.number_particles = n
    particles_sort.run()

n = 4000, simulation time = 0.775725 seconds, absmin = 0.767873, absavg = 0.956887

n = 8000, simulation time = 2.07739 seconds, absmin = 0.765593, absavg = 0.956928

n = 16000, simulation time = 5.54958 seconds, absmin = 0.761189, absavg = 0.956995

n = 32000, simulation time = 15.0914 seconds, absmin = 0.74494, absavg = 0.957091

n = 64000, simulation time = 41.5065 seconds, absmin = 0.738748, absavg = 0.956985



We run the autograder against the results to see how we did,

In [9]:
autograder.autograde_hw2a("particles_sort_summary.txt")


Scaling results:
                                                       Serial Scaling Plots                                                       
                                                                                                                                  
                Simulation time vs number of particles                      Simulation time vs number of particles (log-log)      
     Sim. time (s)                                                   Sim. time (s)                                                
       45 +--------------------------------------------------+         100 +--------------------------------------------------+   
          |                                              O   |             +                                                  |   
       40 +                                                  |             +                                             O    |   
       35 +                                                  |  

Our sorting algorithm scales as $O(n^{1.43})$, not bad considering how simple it is!

Next, we benchmark the naive algorithm using simulations of 500, 1000, 2000, and 4000 particles,

In [4]:
particles_naive.summary_filename = b"particles_naive_summary.txt"
for n in [500, 1000, 2000, 4000]:
    particles_naive.number_particles = n
    particles_naive.run()

n = 500, simulation time = 0.660938 seconds, absmin = 0.798344, absavg = 0.957176

n = 1000, simulation time = 2.63296 seconds, absmin = 0.781476, absavg = 0.956711

n = 2000, simulation time = 10.4075 seconds, absmin = 0.779835, absavg = 0.95688

n = 4000, simulation time = 41.5172 seconds, absmin = 0.769603, absavg = 0.956527



We run the autograder on the naive results,

In [6]:
autograder.autograde_hw2a("particles_naive_summary.txt")



Scaling results:
                                                       Serial Scaling Plots                                                       
                                                                                                                                  
                Simulation time vs number of particles                      Simulation time vs number of particles (log-log)      
     Sim. time (s)                                                   Sim. time (s)                                                
       45 +--------------------------------------------------+         100 +--------------------------------------------------+   
          |                                            O     |             +                                                  |   
       40 +                                                  |             +                                        O         |   
       35 +                                                  |  

As expected, the scaling is indeed $O(n^2)$.